# Preparation

#### Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Imports

In [1]:
# to import custom python files, append the path to sys here
import sys
sys.path.append('/content/drive/My Drive')
from util import get_classes, process

import math
import torch
import torchvision

#### Constants

In [3]:
DATA_PATH = '/content/drive/My Drive/Colab Notebooks/car_data/car_data'
BATCH_SIZE = 32
NUM_CLASSES = 196
LEARNING_RATE = 0.1
MOMENTUM = 0.9
NUM_EPOCHS = 10
VALIDATION_PERIOD = 20

# Load datasets

In [4]:
# load datasets using ImageFolder structure
train_data = torchvision.datasets.ImageFolder(f'{DATA_PATH}/train', transform=process.DATA_AUGMENT_TRANSFORM)
test_data = torchvision.datasets.ImageFolder(f'{DATA_PATH}/test', transform=process.PREPROCESS_TRANSFORM)

# create a validation set using 15% of the training data
train_split = math.floor(0.85 * len(train_data))
valid_split = math.ceil(0.15 * len(train_data))
train_data, valid_data = torch.utils.data.random_split(train_data, [train_split, valid_split])

# create data loaders using datasets, set shuffle to true so epochs are different
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=True)

# Load Model

In [5]:
# also considering densenet121 and resnet34
model = torchvision.models.resnet101(pretrained=True)
num_input_filters = model.fc.in_features

# define the fully-connected layer to be a linear transformation from num_input_filters to num_classes
model.fc = torch.nn.Linear(num_input_filters, NUM_CLASSES)

# use the gpu
model.to('cuda')

# define the criterion as the cross entropy loss function
criterion = torch.nn.CrossEntropyLoss()

# define the stochastic gradient descent with the learning rate and momentum
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

# lower the learning rate when we stop improving as fast
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=False, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

# Train Model

#### Validation Function

In [8]:
# validation function to calculate the loss
def validation(model, valid_loader, criterion):
    total_loss = 0
    total_accuracy = 0

    # iterate over batches of validation data
    for images, labels in valid_loader:
        # move images and labels to gpu
        images, labels = images.to('cuda'), labels.to('cuda')

        # forward the images through the model
        output = model.forward(images)
  
        # sum the losses
        loss += criterion(output, labels).item()
  
        # calculate a probability tensor by undoing log of loss function
        probability = torch.exp(output)

        # get the prediction
        prediction = probability.max(dim=1)[1]

        # sum average accuracy 
        accuracy += (labels.data == prediction).mean()
    
    loss = total_loss / len(valid_loader)
    accuracy = total_accuracy / len(valid_loader)
    
    return loss, accuracy

#### Training

In [ ]:
steps = 0

model.train()
for epoch in range(NUM_EPOCHS):
    running_loss = 0

    # iterate over batches of training data
    for images, labels in train_loader:
        steps += 1

        # move images and labels to gpu
        images, labels = images.to('cuda'), labels.to('cuda')
        
        # zero out accumulated parameter gradients before doing back propagation 
        optimizer.zero_grad()
        
        # forward the images through the model
        outputs = model.forward(images)

        # back propagate the loss from the loss function to each parameter
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # validation step
        if steps % VALIDATION_PERIOD == 0:
            # set model to eval mode
            model.eval()
            
            # turn gradients off during validation
            with torch.no_grad():
                validation_loss, accuracy = validation(model, valid_loader, criterion)
            
            # print each epoch
            print(f'Epoch #: {epoch + 1}, \
            Running Loss: {running_loss / VALIDATION_PERIOD} \
            Validation Loss: {validation_loss} \
            Validation Accuracy: {accuracy}')
            
            # set model to train mode
            model.train()

            # slow learning rate according to validation loss
            scheduler.step(validation_loss)


# Test Model

In [ ]:
correct = 0
total = 0

# test model using holdout test set with gradients off
with torch.no_grad():
    for images, labels in testloader:
        # move images and labels to gpu
        images, labels = images.to('cuda'), labels.to('cuda')

        # get model's probabilities of labels from images
        outputs = model(images)

        # get prediction
        prediction = torch.max(outputs.data, dim=1)[1]

        # sum number of correct predictions and total
        total += labels.size(0)
        correct += (prediction == labels).sum().item()

print(f"Test accuracy of model: {100 * correct / total}%")

Test accuracy of model: 72.951%


# Save Model

In [ ]:
# save the feature weights state, new fully connected layer, class-to-index map, optimiser state, and number of epochs
checkpoint = {'state_dict': model.state_dict(),
              'model': model.fc,
              'class_to_idx': train_data.class_to_idx,
              'opt_state': optimizer.state_dict,
              'num_epochs': epochs
              }

torch.save(checkpoint, 'checkpoint.pth')